In [1]:
!pip install flask jupyter_client


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install flask-cors


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import jupyter_client
from flask import Flask, request, jsonify
from flask_cors import CORS

In [12]:
app = Flask(__name__)
CORS(app)

In [13]:
@app.route('/send-gen-ppt', methods=['POST'])
def receive_message():
    data = request.json  # Assuming JSON payload
    
    print(data['searchQuery'])



@app.route("/revert-gen-ppt", methods=['GET'])
def return_home():
  
    return jsonify({
        "message" : "Message-Gone"
    })

app.run(debug=True, use_reloader=False, port=8443)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8443
Press CTRL+C to quit
127.0.0.1 - - [11/May/2024 19:07:03] code 400, message Bad request version ('\x00\x02\x01\x00\x003\x04ï\x04í')
127.0.0.1 - - [11/May/2024 19:07:03] "\x16\x03\x01\x06ò\x01\x00\x06î\x03\x03¯Ï³Ñ\x18ðGK¼0\x17\x06Þ\x08;ÖñYû=JõÔ³\x17Ò` #¡== h\x81Äpü\x1aÙ\x95°{MU ö\x80~Ä\x83ö=È§Ô\x14CNT\x11ý@0ñ\x00 ºº\x13\x01\x13\x02\x13\x03À+À/À,À0Ì©Ì¨À\x13À\x14\x00\x9c\x00\x9d\x00/\x005\x01\x00\x06\x85jj\x00\x00\x00\x17\x00\x00\x00#\x00\x00\x00\x0d\x00\x12\x00\x10\x04\x03\x08\x04\x04\x01\x05\x03\x08\x05\x05\x01\x08\x06\x06\x01\x00\x0b\x00\x02\x01\x00\x003\x04ï\x04í" HTTPStatus.BAD_REQUEST -
127.0.0.1 - - [11/May/2024 19:07:04] code 400, message Bad request version ('\x02h2\x08http/1.1\x00')
127.0.0.1 - - [11/May/2024 19:07:04] "\x16\x03\x01\x06Ò\x01\x00\x06Î\x03\x03R4\x0e.%\x05\x13Òá­O\x84\x841CàqËýÿ|7Ù\x8cFû?G\x09æ¥h \x98>\x0d\x0dÓÂ7Üo<\x8f\x16Hua\x8f¦\x12Ù³ë±\x85Wæ½g\x84Þ\x80]ô\x00 **\x13\x01\x13\x02\x13\x03À+À/À,À0Ì©Ì¨À\x13À\x14\x00\x9c\x00\x9d\x00